In [2]:
import pandas as pd

df = pd.read_csv('AAA_final.csv')

df.head()

publisher                 game_names      video_type   video_link  \
0    Capcom              Lost Planet 3         Trailer  igfmUe-IBeQ   
1    Capcom              Lost Planet 3  Gameplay video  8anIGeV18HE   
2    Capcom              Lost Planet 3  Gameplay video  afZAmzYWEu4   
3    Capcom  Call of Duty: Finest Hour  Gameplay Video  2f8S7gPK4b4   
4    Capcom  Call of Duty: Finest Hour         Trailer  qDCADmoCec8   

                                     video_url game_release_date  \
0  https://www.youtube.com/watch?v=igfmUe-IBeQ        2013-08-26   
1  https://www.youtube.com/watch?v=8anIGeV18HE        2013-08-26   
2  https://www.youtube.com/watch?v=afZAmzYWEu4        2013-08-26   
3  https://www.youtube.com/watch?v=2f8S7gPK4b4        2004-11-16   
4  https://www.youtube.com/watch?v=qDCADmoCec8        2004-11-16   

  youtube_uploaded_date  days_before_launch  viewCount  likeCount  \
0   2012-08-14 11:14:55              -377.0      42169        208   
1   2016-02-28 19:27:03               916.0       1382          6   
2   2016-02-29 13:49:12               917.0         88          0   
3   2019-12-16 17:30:00              5508.0       4397          0   
4   2011-05-19 22:57:07              2375.0       3144         38   

   commentCount  subscriberCount  
0            38           336000  
1             1             3600  
2             0             3600  
3             8               41  
4             6         19200000

In [8]:
# 1) datetime 타입으로 변환
df['youtube_uploaded_date'] = pd.to_datetime(df['youtube_uploaded_date'])

# 2) 2015년 1월 1일 기준 이후 레코드만 슬라이싱
#df_after_2015 = df[df['youtube_uploaded_date'] >= '2020-01-01']

# 또는 연도 기준 필터링
df_after_2020 = df[df['youtube_uploaded_date'].dt.year >= 2020]

print(df_after_2020.shape)
# 3) 결과 확인
df_after_2020.head()

(1309, 12)


publisher                                       game_names      video_type  \
10    Capcom                           Street Fighter Alpha 3  Gameplay Video   
23    Capcom                          Resident Evil: Dead Aim  Gameplay Video   
27    Capcom                                Breath of Fire IV  Gameplay Video   
31    Capcom                      Mega Man X: Command Mission  Gameplay Video   
44    Capcom  The Nightmare Before Christmas: Oogie's Revenge  Gameplay Video   

     video_link                                    video_url  \
10  mDQSK4aSoWM  https://www.youtube.com/watch?v=mDQSK4aSoWM   
23  --tL-EM4IHc  https://www.youtube.com/watch?v=--tL-EM4IHc   
27  Vo3OUPZ04aU  https://www.youtube.com/watch?v=Vo3OUPZ04aU   
31  G_ZmjNShSHk  https://www.youtube.com/watch?v=G_ZmjNShSHk   
44  NPbcaNsGaVs  https://www.youtube.com/watch?v=NPbcaNsGaVs   

   game_release_date youtube_uploaded_date  days_before_launch  viewCount  \
10        1998-06-29   2022-10-29 21:19:44              8888.0        369   
23        2003-02-13   2022-02-04 23:00:00              6931.0      13524   
27        2000-04-27   2025-02-19 00:00:21              9064.0         26   
31        2004-07-29   2020-06-29 16:00:13              5814.0      85709   
44        2004-10-21   2024-05-11 20:37:46              7142.0         89   

    likeCount  commentCount  subscriberCount  
10          6             0              958  
23        318            58          1350000  
27          1             0              317  
31        841            82           561000  
44          7             1             1790

In [10]:
# Cell 1: 필요한 라이브러리 설치 & 임포트
!pip install google-api-python-client isodate --quiet

import pandas as pd
import isodate
from googleapiclient.discovery import build


In [ ]:
# Cell 2: API 키로 YouTube 클라이언트 초기화
API_KEY = 'API_KEY'
youtube = build('youtube', 'v3', developerKey=API_KEY)

In [12]:
# Cell 3: 한 번에 50개씩 받아올 함수 정의
def get_duration_batch(video_ids):
    """
    video_ids: list of up to 50 video IDs
    returns: dict {video_id: duration_in_seconds}
    """
    resp = youtube.videos().list(
        part='contentDetails',
        id=','.join(video_ids)
    ).execute()
    
    durations = {}
    for item in resp.get('items', []):
        vid = item['id']
        iso_dur = item['contentDetails']['duration']  # ex. 'PT15M33S'
        seconds = int(isodate.parse_duration(iso_dur).total_seconds())
        durations[vid] = seconds
    return durations


In [13]:
# Cell 4: df_after_2020에서 ID 뽑아서 배치로 호출
video_ids = df_after_2020['video_link'].astype(str).tolist()
all_durations = {}

for i in range(0, len(video_ids), 50):
    batch = video_ids[i:i+50]
    durs = get_duration_batch(batch)
    all_durations.update(durs)
    print(f"Processed {i+len(batch)} / {len(video_ids)}")


Processed 50 / 1309
Processed 100 / 1309
Processed 150 / 1309
Processed 200 / 1309
Processed 250 / 1309
Processed 300 / 1309
Processed 350 / 1309
Processed 400 / 1309
Processed 450 / 1309
Processed 500 / 1309
Processed 550 / 1309
Processed 600 / 1309
Processed 650 / 1309
Processed 700 / 1309
Processed 750 / 1309
Processed 800 / 1309
Processed 850 / 1309
Processed 900 / 1309
Processed 950 / 1309
Processed 1000 / 1309
Processed 1050 / 1309
Processed 1100 / 1309
Processed 1150 / 1309
Processed 1200 / 1309
Processed 1250 / 1309
Processed 1300 / 1309
Processed 1309 / 1309


In [17]:
# Cell 5 수정
df_after_2020.loc[:, 'duration_sec'] = df_after_2020['video_link'].map(all_durations)


In [18]:
# Cell 6: 확인
df_after_2020.head()


publisher                                       game_names      video_type  \
10    Capcom                           Street Fighter Alpha 3  Gameplay Video   
23    Capcom                          Resident Evil: Dead Aim  Gameplay Video   
27    Capcom                                Breath of Fire IV  Gameplay Video   
31    Capcom                      Mega Man X: Command Mission  Gameplay Video   
44    Capcom  The Nightmare Before Christmas: Oogie's Revenge  Gameplay Video   

     video_link                                    video_url  \
10  mDQSK4aSoWM  https://www.youtube.com/watch?v=mDQSK4aSoWM   
23  --tL-EM4IHc  https://www.youtube.com/watch?v=--tL-EM4IHc   
27  Vo3OUPZ04aU  https://www.youtube.com/watch?v=Vo3OUPZ04aU   
31  G_ZmjNShSHk  https://www.youtube.com/watch?v=G_ZmjNShSHk   
44  NPbcaNsGaVs  https://www.youtube.com/watch?v=NPbcaNsGaVs   

   game_release_date youtube_uploaded_date  days_before_launch  viewCount  \
10        1998-06-29   2022-10-29 21:19:44              8888.0        369   
23        2003-02-13   2022-02-04 23:00:00              6931.0      13524   
27        2000-04-27   2025-02-19 00:00:21              9064.0         26   
31        2004-07-29   2020-06-29 16:00:13              5814.0      85709   
44        2004-10-21   2024-05-11 20:37:46              7142.0         89   

    likeCount  commentCount  subscriberCount  duration_sec  
10          6             0              958          8704  
23        318            58          1350000          5941  
27          1             0              317          6292  
31        841            82           561000         29346  
44          7             1             1790           868

In [19]:
df_after_2020.shape

(1309, 13)

In [20]:
# Cell 7: ‘duration_sec’을 분 단위로 반올림해 ‘duration_min’ 컬럼 생성
# (초 데이터를 분으로 나눈 뒤, .round()로 가장 가까운 정수로 반올림)
df_after_2020 = df_after_2020.copy()  # 안전하게 복사본을 만든 뒤
df_after_2020.loc[:, 'duration_min'] = (df_after_2020['duration_sec'] / 60).round().astype(int)

# 확인
df_after_2020[['video_link', 'duration_sec', 'duration_min']].head()

video_link  duration_sec  duration_min
10  mDQSK4aSoWM          8704           145
23  --tL-EM4IHc          5941            99
27  Vo3OUPZ04aU          6292           105
31  G_ZmjNShSHk         29346           489
44  NPbcaNsGaVs           868            14

In [23]:
# 맞는 변수명으로 삭제하기
del df_after_2020['duration_sec']

In [24]:
# DataFrame을 CSV 파일로 저장하기
df_after_2020.to_csv('AAA_final_final.csv', index=False, encoding='utf-8-sig')